## Import

In [16]:
import pandas as pd

In [17]:
import os
import sys
currentdir = os.path.dirname(os.path.realpath('merge_DK'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from func import quarter_to_month, GT_dict, reindex

## Target

In [18]:
# Importing AUS monthly regional data
df_target_DK = pd.read_csv('data/df_target_DK.csv', sep = ',', index_col = None, parse_dates = ['date'])

In [19]:
df_target_DK = df_target_DK.rename(columns = {'target_actual_lag' : 'target_lag'})

In [20]:
df_target_DK = df_target_DK.sort_values(by = 'date')

In [21]:
df_target_DK['target_lag'] =  df_target_DK.groupby(['ID'])['target_actual'].shift(1)

In [22]:
df_target_DK.date.max()

Timestamp('2019-09-01 00:00:00')

In [23]:
df_target_DK.date.min()

Timestamp('2007-01-01 00:00:00')

## Job posts

In [24]:
df_job_posts_DK = pd.read_csv("data/df_jobposts_DK.csv", parse_dates = ['date']) 

In [25]:
df_job_posts_DK.drop(['labour_force'], axis = 1, inplace = True)

#### Merge on labour force

Note: This has been shiftet one month - check control.ipynb

In [26]:
df_labour_force_DK = pd.read_csv("data/df_labour_force_DK.csv", parse_dates = ['date']) 

In [27]:
df_job_posts_DK = df_job_posts_DK.merge(df_labour_force_DK, on = ['date', 'ID'])

#### Create job rate

In [28]:
jobindex_list = ['jobs', 'sector_information_technology', 'sector_engineering_technology','sector_management_staff', 'sector_trade_service',
       'sector_industry_craft', 'sector_sales_communication',
       'sector_teaching', 'sector_office_finance', 'sector_social_health',
       'sector_other']

In [29]:
for col in jobindex_list:
    df_job_posts_DK[col] = (df_job_posts_DK[col] / df_job_posts_DK['labour_force'])*100

In [30]:
df_job_posts_DK.drop(['labour_force'], axis = 1, inplace = True)

In [31]:
df_job_posts_DK.columns

Index(['date', 'jobs', 'ID', 'sector_information_technology',
       'sector_engineering_technology', 'sector_management_staff',
       'sector_trade_service', 'sector_industry_craft',
       'sector_sales_communication', 'sector_teaching',
       'sector_office_finance', 'sector_social_health', 'sector_other'],
      dtype='object')

In [32]:
df_job_posts_DK.dropna(inplace=True)

In [33]:
df_job_posts_DK.date.max()

Timestamp('2019-09-01 00:00:00')

In [34]:
df_job_posts_DK.date.min()

Timestamp('2007-02-01 00:00:00')

New september values

In [37]:
df_job_post_sep= pd.read_csv("data/df_DK_jobposts_sep_final.csv", parse_dates = ['date']) 

In [42]:
df_job_posts_DK = df_job_posts_DK[df_job_posts_DK.date < df_job_posts_DK.date.max()]

In [45]:
df_job_posts_DK = pd.concat([df_job_posts_DK, df_job_post_sep])

## Controls

In [46]:
df_controls_DK = pd.read_csv('data/df_controls.csv', sep = ',', index_col = None, parse_dates = ['date'])

In [47]:
df_controls_DK.date.max()

Timestamp('2019-09-01 00:00:00')

In [48]:
df_controls_DK.date.min()

Timestamp('2007-02-01 00:00:00')

## Google

In [49]:
dfTrends = pd.read_csv('data/dfTrends_DK.csv', sep = ';', index_col = 0, parse_dates=['date'])

In [50]:
dfTrends.drop(['geo', 'trends'], axis=1, inplace=True)

### Cross sectional GT

In [51]:
# dfTrends_cross = pd.read_csv('data/dfTrends_DK_cross.csv', sep = ';', index_col=0, parse_dates=['date'])

In [52]:
# dfTrends_cross.drop(['geoCode', 'geoName', 'trends'], axis = 1, inplace=True)

In [53]:
# dfTrends_cross.columns

## Merge


In [54]:
df_DK = df_target_DK.merge(df_job_posts_DK, on = ['date', 'ID'], how = 'left')

In [55]:
df_DK = df_DK.merge(df_controls_DK, on = ['date', 'ID'], how = 'left')

In [56]:
df_DK = pd.merge(df_DK, dfTrends, how = 'left', left_on=['ID', 'date'], right_on=['ID', 'date'])

In [57]:
# df_DK = pd.merge(df_DK, dfTrends_cross, how = 'left', on = ['ID', 'date'])

In [58]:
df_DK.sort_values(['ID', 'date'], inplace=True)

In [59]:
df_DK.dropna(inplace=True)

### Reindexing GT


In [60]:
#Reindexing trends
df_DK = df_DK.apply(lambda col: reindex(col) if col.name in GT_dict() else col).copy()

## Check result

In [61]:
df_DK.ID.value_counts()

North Denmark       152
Central Denmark     152
Zealand             152
Southern Denmark    152
Capital             152
Name: ID, dtype: int64

In [62]:
df_DK.date.min()

Timestamp('2007-02-01 00:00:00')

In [63]:
df_DK.date.max()

Timestamp('2019-09-01 00:00:00')

## Save

In [64]:
df_DK.to_csv('data/df_DK.csv', index = False)

In [65]:
df_DK.head()

,date,target_actual,ID,target_lag,jobs,sector_information_technology,sector_engineering_technology,sector_management_staff,sector_trade_service,sector_industry_craft,...,GT_DK_12,GT_DK_13,GT_DK_14,GT_DK_15,GT_DK_16,GT_DK_17,GT_DK_18,GT_DK_19,GT_DK_20,GT_DK_21
9,2007-02-01,4.8,Capital,4.9,1.260650,0.088754,0.089670,0.126546,0.184036,0.122309,...,29.35,46.46,62.0,18.0,24.0,32.0,15.0,79.0,9.0,18.0
13,2007-03-01,4.6,Capital,4.8,1.477047,0.100425,0.110088,0.136776,0.225928,0.150005,...,43.48,20.20,44.0,20.0,21.0,54.0,16.0,85.0,6.0,22.0
15,2007-04-01,4.3,Capital,4.6,1.369270,0.081074,0.102986,0.146464,0.210355,0.138738,...,13.04,38.38,72.0,19.0,0.0,59.0,25.0,99.0,10.0,36.0
23,2007-05-01,4.3,Capital,4.3,1.407431,0.080941,0.111567,0.144036,0.218530,0.144151,...,23.91,39.39,35.0,19.0,20.0,51.0,21.0,76.0,10.0,10.0
27,2007-06-01,4.1,Capital,4.3,1.342219,0.078108,0.107819,0.113041,0.208325,0.150760,...,16.30,43.43,33.0,21.0,0.0,43.0,30.0,89.0,9.0,11.0
